In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from config import mysqlpassword

In [2]:
stats_csv = "Resources/seasons_stats.csv"
stats_df = pd.read_csv(stats_csv, index_col = 0)
stats_df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [3]:
stats_df.dropna(axis=1, how="all", inplace=True)
stats_df.dropna(how="any", inplace=True)
stats_df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
5730,1980.0,Tiny Archibald*,PG,31.0,BOS,80.0,80.0,2864.0,15.3,0.574,...,0.830,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0
5748,1980.0,Larry Bird*,PF,23.0,BOS,82.0,82.0,2955.0,20.5,0.538,...,0.836,216.0,636.0,852.0,370.0,143.0,53.0,263.0,279.0,1745.0
5781,1980.0,M.L. Carr,SF,29.0,BOS,82.0,7.0,1994.0,14.6,0.526,...,0.739,106.0,224.0,330.0,156.0,120.0,36.0,143.0,214.0,914.0
5786,1980.0,Don Chaney,SG,33.0,BOS,60.0,0.0,523.0,7.8,0.402,...,0.762,31.0,42.0,73.0,38.0,31.0,11.0,33.0,80.0,167.0
5802,1980.0,Dave Cowens*,C,31.0,BOS,66.0,55.0,2159.0,14.8,0.477,...,0.779,126.0,408.0,534.0,206.0,69.0,61.0,108.0,216.0,940.0


In [4]:
# Create foreign key to be used for joins based on names (index remains primary key)
stats_df['namekey'] = stats_df['Player'].str.lower()
for s in [" ", "'", "-", ".", ",", "*"]:
    stats_df['namekey'] = stats_df['namekey'].str.replace(s,'')
stats_df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,namekey
5730,1980.0,Tiny Archibald*,PG,31.0,BOS,80.0,80.0,2864.0,15.3,0.574,...,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0,tinyarchibald
5748,1980.0,Larry Bird*,PF,23.0,BOS,82.0,82.0,2955.0,20.5,0.538,...,216.0,636.0,852.0,370.0,143.0,53.0,263.0,279.0,1745.0,larrybird
5781,1980.0,M.L. Carr,SF,29.0,BOS,82.0,7.0,1994.0,14.6,0.526,...,106.0,224.0,330.0,156.0,120.0,36.0,143.0,214.0,914.0,mlcarr
5786,1980.0,Don Chaney,SG,33.0,BOS,60.0,0.0,523.0,7.8,0.402,...,31.0,42.0,73.0,38.0,31.0,11.0,33.0,80.0,167.0,donchaney
5802,1980.0,Dave Cowens*,C,31.0,BOS,66.0,55.0,2159.0,14.8,0.477,...,126.0,408.0,534.0,206.0,69.0,61.0,108.0,216.0,940.0,davecowens


In [5]:
# stats_df.set_index("namekey", verify_integrity=True, inplace=True)

In [6]:
#give the primary key column a name
stats_df.index.name="id"
stats_df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,namekey
id,,,,,,,,,,,,,,,,,,,,,
5730,1980.0,Tiny Archibald*,PG,31.0,BOS,80.0,80.0,2864.0,15.3,0.574,...,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0,tinyarchibald
5748,1980.0,Larry Bird*,PF,23.0,BOS,82.0,82.0,2955.0,20.5,0.538,...,216.0,636.0,852.0,370.0,143.0,53.0,263.0,279.0,1745.0,larrybird
5781,1980.0,M.L. Carr,SF,29.0,BOS,82.0,7.0,1994.0,14.6,0.526,...,106.0,224.0,330.0,156.0,120.0,36.0,143.0,214.0,914.0,mlcarr
5786,1980.0,Don Chaney,SG,33.0,BOS,60.0,0.0,523.0,7.8,0.402,...,31.0,42.0,73.0,38.0,31.0,11.0,33.0,80.0,167.0,donchaney
5802,1980.0,Dave Cowens*,C,31.0,BOS,66.0,55.0,2159.0,14.8,0.477,...,126.0,408.0,534.0,206.0,69.0,61.0,108.0,216.0,940.0,davecowens


In [7]:
# Rename Columns (replace all % with prc, and / with -)

rename_dict = {}
for c in list(stats_df.columns):
    rename_dict[c] = c.replace('%','prc').replace("/", '')

stats_df.rename(columns=rename_dict, inplace=True)
stats_df.columns

Index(['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TSprc',
       '3PAr', 'FTr', 'ORBprc', 'DRBprc', 'TRBprc', 'ASTprc', 'STLprc',
       'BLKprc', 'TOVprc', 'USGprc', 'OWS', 'DWS', 'WS', 'WS48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FGprc', '3P', '3PA', '3Pprc', '2P',
       '2PA', '2Pprc', 'eFGprc', 'FT', 'FTA', 'FTprc', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'namekey'],
      dtype='object')

In [8]:
# select desired columns
stats_df = stats_df[['Year', 'Player', 'Pos', 'Age', 'G', 'GS', 'MP', 'PER', 'TSprc',
       '3PAr', 'FTr', 'ORBprc', 'DRBprc', 'TRBprc', 'ASTprc', 'STLprc',
       'BLKprc', 'TOVprc', 'USGprc', 'OWS', 'DWS', 'WS', 'WS48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FGprc', '3P', '3PA', '3Pprc', '2P',
       '2PA', '2Pprc', 'eFGprc', 'FT', 'FTA', 'FTprc', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'namekey']]
stats_df

,Year,Player,Pos,Age,G,GS,MP,PER,TSprc,3PAr,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,namekey
id,,,,,,,,,,,,,,,,,,,,,
5730,1980.0,Tiny Archibald*,PG,31.0,80.0,80.0,2864.0,15.3,0.574,0.023,...,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0,tinyarchibald
5748,1980.0,Larry Bird*,PF,23.0,82.0,82.0,2955.0,20.5,0.538,0.098,...,216.0,636.0,852.0,370.0,143.0,53.0,263.0,279.0,1745.0,larrybird
5781,1980.0,M.L. Carr,SF,29.0,82.0,7.0,1994.0,14.6,0.526,0.054,...,106.0,224.0,330.0,156.0,120.0,36.0,143.0,214.0,914.0,mlcarr
5786,1980.0,Don Chaney,SG,33.0,60.0,0.0,523.0,7.8,0.402,0.032,...,31.0,42.0,73.0,38.0,31.0,11.0,33.0,80.0,167.0,donchaney
5802,1980.0,Dave Cowens*,C,31.0,66.0,55.0,2159.0,14.8,0.477,0.013,...,126.0,408.0,534.0,206.0,69.0,61.0,108.0,216.0,940.0,davecowens
5835,1980.0,Chris Ford,SG,31.0,73.0,73.0,2115.0,13.0,0.537,0.231,...,77.0,104.0,181.0,215.0,111.0,27.0,105.0,178.0,816.0,chrisford
5867,1980.0,Gerald Henderson,PG,24.0,76.0,2.0,1061.0,12.7,0.539,0.016,...,37.0,46.0,83.0,147.0,45.0,15.0,109.0,96.0,473.0,geraldhenderson
5901,1980.0,Jeff Judkins,SF,23.0,65.0,0.0,674.0,14.1,0.567,0.098,...,32.0,34.0,66.0,47.0,29.0,5.0,49.0,91.0,351.0,jeffjudkins
5947,1980.0,Pete Maravich*,SG,32.0,43.0,4.0,964.0,13.8,0.500,0.028,...,17.0,61.0,78.0,83.0,24.0,6.0,82.0,79.0,589.0,petemaravich


Connect To Database

In [9]:
# Create Database Connection
rds_connection_string = f"root:{mysqlpassword}@localhost/basketball_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# Confirm tables
engine.table_names()

['player_data', 'player_stats', 'players', 'salary']

Load DataFrames into Database

In [14]:
stats_df.to_sql(name='player_stats', con=engine, if_exists='append')